# MLOps Workshop: Testing & Monitoring Integration

## Einführung
In diesem Notebook werden wir uns mit der Integration von Tests und Monitoring für unser Customer Churn Prediction Projekt beschäftigen. Wir werden verschiedene Testebenen implementieren und ein robustes Monitoring-System aufbauen.

## Lernziele
Nach Abschluss dieses Notebooks können Sie:
- Unit-Tests mit pytest implementieren
- Integrationstests für ML-Pipelines erstellen
- Performance-Monitoring aufsetzen
- Logging-Strategien entwickeln
- Metriken für Modell-Drift erkennen

## Voraussetzungen
- Abgeschlossenes API-Development (Notebook 05)
- pytest und monitoring-tools installiert
- Grundlegendes Verständnis von Testing-Konzepten

## 1. Setup und Installation


In [ ]:
# Benötigte Bibliotheken importieren
import pytest
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import logging
import prometheus_client
from prometheus_client import Counter, Gauge, Histogram
import time
import warnings
warnings.filterwarnings('ignore')

# Logging konfigurieren
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# MLflow Setup
mlflow.set_tracking_uri("file:./mlruns")


## 2. Unit-Tests implementieren

### Aufgabe 1: Test-Klassen erstellen
Implementieren Sie Unit-Tests für die Kernfunktionen.


In [ ]:
# test_model.py
import pytest
from src.model import ChurnPredictor
from src.data_processor import DataProcessor

class TestChurnPredictor:
    @pytest.fixture
    def model(self):
        """Fixture für das Modell"""
        return ChurnPredictor()
    
    @pytest.fixture
    def sample_data(self):
        """Fixture für Testdaten"""
        return pd.DataFrame({
            'tenure': [12],
            'MonthlyCharges': [89.9],
            'TotalCharges': [1078.8],
            'gender': [1],
            'CustomerAge': [42],
            'AvgCostPerService': [29.97]
        })
    
    def test_predict_proba(self, model, sample_data):
        """Test der Wahrscheinlichkeitsvorhersage"""
        proba = model.predict_proba(sample_data)
        assert isinstance(proba, np.ndarray)
        assert proba.shape[1] == 2
        assert np.all((proba >= 0) & (proba <= 1))
    
    def test_predict(self, model, sample_data):
        """Test der Klassenvorhersage"""
        prediction = model.predict(sample_data)
        assert isinstance(prediction, np.ndarray)
        assert prediction.shape[0] == len(sample_data)
        assert np.all(np.isin(prediction, [0, 1]))
